# Create catalog for spectral extraction

Pick a few interesting objects and create catalogs for each filter with matching IDs for easy analysis after the extraction of the 2D spectra.

In [ ]:
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from jdaviz import Imviz

## Read catalogs

In [ ]:
data_dir = '/Users/cpacifici/DATA/NGDEEP/image3/'
file_f115w = data_dir + 'jw02079-o004_t001_niriss_clear-f115w_cat.ecsv'
file_f150w = data_dir + 'jw02079-o004_t001_niriss_clear-f150w_cat.ecsv'
file_f200w = data_dir + 'jw02079-o004_t001_niriss_clear-f200w_cat.ecsv'

In [ ]:
cat_f115w = Table.read(file_f115w)
cat_f150w = Table.read(file_f150w)
cat_f200w = Table.read(file_f200w)
print(len(cat_f150w))
cat_f150w['sky_centroid']

### Plot in Imviz

In [ ]:
imagefile = data_dir + 'jw02079-o004_t001_niriss_clear-f150w_i2d.fits'
imviz = Imviz()
imviz.load_data(imagefile, data_label='F150W')
imviz.show()

In [ ]:
# Plot markers for all objects
viewer = imviz.default_viewer
viewer.reset_markers()
t_f115w = Table({'coord': cat_f115w['sky_centroid']})
t_f150w = Table({'coord': cat_f150w['sky_centroid']})
t_f200w = Table({'coord': cat_f200w['sky_centroid']})
viewer.marker = {'color': 'green', 'alpha': 0.8, 'markersize': 30, 'fill': False}
viewer.add_markers(t_f115w, use_skycoord=True, marker_name='full_cat_f115w')
viewer.marker = {'color': 'orange', 'alpha': 0.8, 'markersize': 5, 'fill': True}
viewer.add_markers(t_f150w, use_skycoord=True, marker_name='full_cat_f150w')
viewer.marker = {'color': 'red', 'alpha': 0.8, 'markersize': 5, 'fill': True}
viewer.add_markers(t_f200w, use_skycoord=True, marker_name='full_cat_f200w')

## Rewrite catalogs with matching IDs

### Match catalogs and find object in all three

1. Match F115W to F150W -> f115w_base_matches
2. Match f115w_base_matches to F200W -> f115w_base_matches, f200w_try_matches
3. Rematch the matched F115W to F150W -> f115w_matches, f150w_try_matches

In [ ]:
f115w_base = cat_f115w['sky_centroid']
f150w_try = cat_f150w['sky_centroid']

max_sep = 1.0 * u.arcsec
idx, d2d, d3d = f115w_base.match_to_catalog_3d(f150w_try)
sep_constraint = d2d < max_sep
f115w_base_matches = cat_f115w[sep_constraint]
f150w_try_matches = cat_f150w[idx[sep_constraint]]

In [ ]:
f115w_base = f115w_base_matches['sky_centroid']
f200w_try = cat_f200w['sky_centroid']

max_sep = 1.0 * u.arcsec
idx, d2d, d3d = f115w_base.match_to_catalog_3d(f200w_try)
sep_constraint = d2d < max_sep
f115w_base_matches = f115w_base_matches[sep_constraint]
f200w_try_matches = cat_f200w[idx[sep_constraint]]

In [ ]:
f115w_base = f115w_base_matches['sky_centroid']
f150w_try = cat_f150w['sky_centroid']

max_sep = 1.0 * u.arcsec
idx, d2d, d3d = f115w_base.match_to_catalog_3d(f150w_try)
sep_constraint = d2d < max_sep
f115w_matches = f115w_base_matches[sep_constraint]
f150w_try_matches = cat_f150w[idx[sep_constraint]]

In [ ]:
f115w_matches

In [ ]:
# Check that they all have the same length
print("Length F115W match: ", len(f115w_matches))
print("Length F150W match: ", len(f150w_try_matches))
print("Length F200W match: ", len(f200w_try_matches))

### Rewrite catalogs

In [ ]:
f115w_out_file = data_dir + 'jw02079-o004_t001_niriss_clear-f115w_cat_matched.ecsv'
f150w_out_file = data_dir + 'jw02079-o004_t001_niriss_clear-f150w_cat_matched.ecsv'
f200w_out_file = data_dir + 'jw02079-o004_t001_niriss_clear-f200w_cat_matched.ecsv'

This is the critical step. Use the same `label` for the same targets in the three catalogs.

In [ ]:
f150w_try_matches['label'] = f115w_matches['label']
f200w_try_matches['label'] = f115w_matches['label']

In [ ]:
f115w_matches.write(f115w_out_file, format='ascii.ecsv', overwrite=True)
f150w_try_matches.write(f150w_out_file, format='ascii.ecsv', overwrite=True)
f200w_try_matches.write(f200w_out_file, format='ascii.ecsv', overwrite=True)